In [1]:
## Here we will write out the matrix elements for our non-eq setup

from qutip import *
import numpy as np
from scipy import integrate
from helper_code_qutip import * 
import scipy.io

In [2]:
## Firstly, we have to define the integral function C and D

def integral1(i,k,tb,beta,mu,gamma,eigenergies,limit_value = 700,b=50):
    freq=eigenergies[k]-eigenergies[i]
    if( np.absolute(freq) >= 1/10**10):
        integral = (-1.0j/(2*np.pi))*integrate.quad(func1,0,b,args=(tb,beta,mu,gamma),limit=limit_value,weight='cauchy',wvar=eigenergies[k]-eigenergies[i])[0]
    else:
        integral = (-1.0j/(2*np.pi))*integrate.quad(func2,0,b,args=(tb,beta,mu,gamma),limit=limit_value)[0]
    return integral

def integral2(i,k,tb,gamma,eigenergies,limit_value = 700,b=50):
    freq=eigenergies[k]-eigenergies[i]
    if( np.absolute(freq) >= 1/10**10):
        integral = (-1.0j/(2*np.pi))*integrate.quad(spectral_bath,0,b,args=(tb,gamma),limit=limit_value,weight='cauchy',wvar=eigenergies[k]-eigenergies[i])[0]
    else:
        integral = (-1.0j/(2*np.pi))*integrate.quad(spectral_bath_2,0,b,args=(tb,gamma),limit=limit_value)[0]
    return integral

def C(i,k,tb,beta,mu,gamma,eigenergies):
    val = integral1(i,k,tb,beta,mu,gamma,eigenergies) + 0.5*(func1(eigenergies[k]-eigenergies[i],tb,beta,mu,gamma))

    return val

def D(i,k,tb,beta,mu,gamma,eigenergies):
    val = integral1(i,k,tb,beta,mu,gamma,eigenergies) + integral2(i,k,tb,gamma,eigenergies) + 0.5*(spectral_bath(eigenergies[k]-eigenergies[i],tb,gamma)+func1(eigenergies[k]-eigenergies[i],tb,beta,mu,gamma))
    return val

In [3]:
NL = 1
NM = 2

N = NL + NM
dL = 2**NL
dM = 2**NM
d = 2**N
dims = [2]*N

In [4]:
## Define the Hamiltonian 

w0list = np.linspace(1,1,N)
glist = np.linspace(0.0016,0.0016,N-1)

delta = 1

H_S = create_hamiltonian(w0list,glist,delta,N)

create_sm_list = [create_sm(N,i + 1) for i in range(NL)]  # Create list of sigma_- operators

eigenergies,eigstates=H_S.eigenstates()

In [6]:
## Let us compute the thermal density matrix
beta = 1

rho_th = (-beta*H_S).expm()/((-beta*H_S).expm()).tr() 
print(rho_th)

Quantum object: dims=[[2, 2, 2], [2, 2, 2]], shape=(8, 8), type='oper', dtype=Dense, isherm=True
Qobj data =
[[1.95012772e-02 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 5.28408768e-02 1.68820262e-04 0.00000000e+00
  2.70256017e-07 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 1.68820262e-04 5.26723268e-02 0.00000000e+00
  1.68820262e-04 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 1.43636395e-01
  0.00000000e+00 4.58901050e-04 7.34632021e-07 0.00000000e+00]
 [0.00000000e+00 2.70256017e-07 1.68820262e-04 0.00000000e+00
  5.28408768e-02 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 4.58901050e-04
  0.00000000e+00 1.43178229e-01 4.58901050e-04 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 7.34632021e-07
  0.00000000e+00 4.58901050e-04 1.43636395e-01 0.00000000e+00]
 [0.00000000e+00

In [7]:
## Now we set parameters for the  non-eq setup

beta_list = np.linspace(1,1,NL)
mu_list = np.linspace(-0.5,-0.5,NL)

gamma_list = np.linspace(1,1,NL)

tb = 0.01

In [8]:
number = len(eigenergies)

## Now we will write out the matrix elements

A = np.zeros((number,number),dtype=complex)

for i in range(number):
    for k in range(number):
        sum = 0
        vi = eigstates[i]
        vk = eigstates[k]
        proj_i = vi*vi.dag()
        proj_k = vk*vk.dag()
        for y in range(number):
            for l in range(NL):
                proj_y = eigstates[y]*eigstates[y].dag()
                op1 = commutator(proj_k*create_sm_list[l]*proj_y,create_sm_list[l].dag())*C(k,y,tb,beta_list[l],mu_list[l],gamma_list[l],eigenergies)
                sum += vi.dag()*(op1 + op1.dag())*vi

                op2 = commutator(create_sm_list[l].dag(),proj_y*create_sm_list[l]*proj_k)*D(y,k,tb,beta_list[l],mu_list[l],gamma_list[l],eigenergies)
                sum += vi.dag()*(op2 + op2.dag())*vi

        A[i,k] = sum


In [9]:
print(number)  #should be 2^N

8


In [11]:
rho_th_new = np.zeros((number,number),dtype=complex)

for i in range(number):
    for k in range(number):
        vi = eigstates[i]
        vk = eigstates[k]

        rho_th_new[i,k] = vi.dag()*rho_th*vk

#print(rho_th_new)

rho_th_diag = [rho_th_new[i,i] for i in range(number)]
print(rho_th_diag)


[(0.3916936226812361+0j), (0.1440960310223909+0j), (0.14363566070847075+0j), (0.1427193278727581+0j), (0.053009967367539976+0j), (0.05284060659372312+0j), (0.05250350658219412+0j), (0.019501277171687006+0j)]


In [12]:
## Now we use the lin algebra solver to solve the equation Ax = b, where b is all zeroes

b = np.zeros((number),dtype=complex)

x = np.linalg.solve(A,b)

print(x)

[0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]


In [13]:
print(A)

[[ 2.84077906e-01+0.00000000e+00j -4.24802965e-01+0.00000000e+00j
  -6.38616515e-01+0.00000000e+00j -2.13812450e-01+0.00000000e+00j
   0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j
   0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j]
 [-9.47863536e-02+1.08420217e-19j  6.14281954e-01+1.62630326e-19j
   9.54097912e-17-2.16840434e-19j -2.86229374e-17-1.40133131e-17j
  -5.66403953e-01+2.77555756e-17j -2.12872172e-01-4.16333634e-17j
  -7.12708166e-02+2.77555756e-17j  0.00000000e+00+0.00000000e+00j]
 [-1.42039351e-01+2.71050543e-19j  1.00613962e-16+0.00000000e+00j
   7.80655063e-01+5.57279917e-17j -6.93889390e-18+2.79724161e-17j
  -2.11930423e-01-2.77555756e-17j -8.13028269e-32-1.23259516e-32j
  -4.26684987e-01+1.38777878e-17j  0.00000000e+00+0.00000000e+00j]
 [-4.72522007e-02+0.00000000e+00j -1.47451495e-17+0.00000000e+00j
   4.33680869e-18+1.08420217e-19j  4.51009797e-01-5.42101086e-20j
  -7.03290601e-02+0.00000000e+00j -4.22917979e-01+1.38777878e-17j
  -5.66

In [14]:
## Now we can add one more constraint, that the sum of all the elements of x should be 1, which is basically another row in the matrix A consisting of all 1

#A = np.vstack([A,np.ones((1,number))])

#print(A)

print(np.linalg.matrix_rank(A))

q,r = np.linalg.qr(A.T)

print(r)
diag_r = [r[i][i] for i in range(number)]

print(np.linalg.matrix_rank(A))





7
[[-8.45402054e-01+0.00000000e+00j  3.40519050e-01-3.66265094e-18j
   6.37435709e-01+4.90803955e-17j  1.29943871e-01+6.81902119e-20j
  -1.03348133e-01+2.72398379e-20j -4.78104235e-02+1.38008712e-20j
  -1.11338020e-01-5.11652427e-20j  0.00000000e+00+0.00000000e+00j]
 [ 0.00000000e+00+0.00000000e+00j -8.00987564e-01+0.00000000e+00j
   6.63520827e-02-1.69395842e-17j -1.62874812e-01-1.10760666e-17j
   7.20130712e-01+3.36359576e-17j  2.22776834e-01-2.72705920e-17j
   6.36814986e-02+1.54336407e-17j -1.09078752e-01+2.46582393e-18j]
 [ 0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j
  -6.67716411e-01+0.00000000e+00j -2.86452180e-01-5.80289269e-17j
   3.27809953e-01-5.70311577e-18j -2.35045358e-02+2.01284280e-17j
   7.21161089e-01+2.79772499e-17j -7.12979146e-02+4.29940721e-19j]
 [ 0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j
   0.00000000e+00+0.00000000e+00j -7.64717839e-01+0.00000000e+00j
  -1.97621638e-01+4.04850379e-17j  4.39567310e-01+3.05159651e-17j
   5.

In [15]:
print(diag_r)

[(-0.8454020537838208+0j), (-0.8009875636014169+0j), (-0.6677164106560071+0j), (-0.7647178390587316+0j), (-0.6428422960658245+0j), (-0.8196290902899462+0j), (-0.6253179486917773+0j), (1.3064536567755912e-16+0j)]


In [16]:
## Since last element of r is 0, we can remove the last row of  A and add a row of 1's

A_new = A[:-1]
A_new = np.vstack([A_new,np.ones((1,number))])

print(np.linalg.matrix_rank(A_new))
print(A_new.shape)
print(A_new)

8
(8, 8)
[[ 2.84077906e-01+0.00000000e+00j -4.24802965e-01+0.00000000e+00j
  -6.38616515e-01+0.00000000e+00j -2.13812450e-01+0.00000000e+00j
   0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j
   0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j]
 [-9.47863536e-02+1.08420217e-19j  6.14281954e-01+1.62630326e-19j
   9.54097912e-17-2.16840434e-19j -2.86229374e-17-1.40133131e-17j
  -5.66403953e-01+2.77555756e-17j -2.12872172e-01-4.16333634e-17j
  -7.12708166e-02+2.77555756e-17j  0.00000000e+00+0.00000000e+00j]
 [-1.42039351e-01+2.71050543e-19j  1.00613962e-16+0.00000000e+00j
   7.80655063e-01+5.57279917e-17j -6.93889390e-18+2.79724161e-17j
  -2.11930423e-01-2.77555756e-17j -8.13028269e-32-1.23259516e-32j
  -4.26684987e-01+1.38777878e-17j  0.00000000e+00+0.00000000e+00j]
 [-4.72522007e-02+0.00000000e+00j -1.47451495e-17+0.00000000e+00j
   4.33680869e-18+1.08420217e-19j  4.51009797e-01-5.42101086e-20j
  -7.03290601e-02+0.00000000e+00j -4.22917979e-01+1.38777878e-17

In [17]:
print(A.shape)

(8, 8)


In [18]:
print(np.linalg.matrix_rank(A))
print(A.shape)

7
(8, 8)


In [19]:
b[-1] = 1  ## Last element of b is 1

x = np.linalg.solve(A_new,b)
print(x)

[0.5475308 -1.73966794e-17j 0.12217064+4.07089472e-18j
 0.12178031-1.39867096e-17j 0.12100341+1.05737685e-17j
 0.02725995+2.75030571e-18j 0.02717286+5.50661957e-18j
 0.02699951+2.12606556e-18j 0.00608252+6.35573490e-18j]


In [20]:
##Let us check if this is correct

print(np.dot(A_new,x))
print(np.dot(A[-1],x))

[-1.38777878e-17+0.00000000e+00j  0.00000000e+00+0.00000000e+00j
  9.71771830e-18-3.85185989e-34j  0.00000000e+00-3.85185989e-34j
  3.46944695e-18+1.54074396e-33j -2.48497300e-18-1.92592994e-34j
  0.00000000e+00+7.70371978e-34j  1.00000000e+00+3.08148791e-33j]
(4.0766001685454967e-17+6.93639131056905e-18j)


In [23]:
## As we can see, all the checks are completely satisfied. Thus we can arrange the values of x (real values) in a diagonal density matrix and save it in a .mat file
x_real = [np.real(x[i]) for i in range(number)]

rho = np.diag(x_real)

data_dict = {"dm_ness":rho, "dm_th":rho_th_new, "dm_th_check":np.array(rho_th)}

scipy.io.savemat(F'../matlab/ness_data_NL = {NL}_NM = {NM}_1.mat',data_dict)
